**1a. [CSCI 490] Download & Extract Files**

In [ ]:
import requests
import zipfile
from zipfile import ZipFile

In [ ]:
# path to get fg_zip file
fg_url = 'http://faculty.cs.niu.edu/~dakoop/cs503-2021fa/a7/fg.zip'
fg_response = requests.get(fg_url)

# open the zip file and write the contents of the response from request
open("fg.zip", "wb").write(fg_response.content)

# read the zip file and extract all the files in fg.zip to a directory named data
with zipfile.ZipFile('fg.zip', 'r') as fg_zip:
    # extract zip file in local directory 
    fg_zip.extractall()

In [ ]:
# path to get hij_zip file 
hij_url = 'http://faculty.cs.niu.edu/~dakoop/cs503-2021fa/a7/hij.zip'
hij_response = requests.get(hij_url)

# open the zip file and write the contents of the response from request 
open("hij.zip", "wb").write(hij_response.content)
    
# read the zip file and extract all the files in hij.zip to a directory named data
with zipfile.ZipFile('hij.zip', 'r') as hij_zip:
    # extract zip file in local directory 
    hij_zip.extractall()

In [ ]:
# path to get tu.zip file
tu_url = 'http://faculty.cs.niu.edu/~dakoop/cs503-2021fa/a7/tu.zip'
tu_response = requests.get(tu_url)

# ope the zip file and write the contents of the response from request
open("tu.zip", "wb").write(tu_response.content)

# read the zip file and extract all the files in tu.zip to a directory named data
with zipfile.ZipFile('tu.zip', 'r') as tu_zip:
    # extract zip file in local directory 
    tu_zip.extractall()

**2. Find Matching Files**

In [ ]:
import os
from glob import glob

# path to data directory which contains all csv files
PATH = "data"
# look if any csv files exist
EXT = "*.csv"
# get the paths for each csv file from directories and subdirectories
all_csv_files = [file
                 for path, subdir, files in os.walk(PATH)
                 for file in glob(os.path.join(path, EXT))]
# print all the paths to each csv files
print(all_csv_files)

['data/jefferson.csv', 'data/ulster.csv', 'data/g/greene.csv', 'data/g/genesee.csv', 'data/f/franklin.csv', 'data/f/fulton.csv', 'data/t/tompkins.csv', 'data/t/tioga.csv', 'data/h/herkimer.csv', 'data/h/hamilton.csv']


**3. Use Threads to Process Files**

In [ ]:
import pandas as pd
import numpy as np
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor

def process():
    # list of files
    li = []

    # read in all csv files
    for filename in all_csv_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
        
    # concatenate all csv files into one 
    frame = pd.concat(li, axis=0, ignore_index=True)
    display(frame)
    
    # filter only data class that has electricity and replace the value with not a number
    f = frame[frame['data_class'] == 'electricity'].replace(-999.000, np.nan)
    display(f)
    
    frames = [frame, f]

    # concatonate the dataframes to add in the filters
    result = pd.concat(frames)
    display(result)

    # filter only the years that include 2019
    df2019 = result[result['year'] == 2019]
    display(df2019)    
    
    # write a new csv file that is compressed to a gzip file
    df2019 = df2019.to_csv('2019.csv.gz', compression='gzip', index = 0)
    
    # filter only the years that include 2020
    df2020 = result[result['year'] == 2020]
    display(df2020)
    
    # write a new csv file that is compressed to a gzip file
    df2020 = df2020.to_csv('2020.csv.gz', compression='gzip', index = 0)

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(process())

,county_name,com_name,year,month,data_class,data_field_display_name,unit,value,number_of_accounts
0,Jefferson,West Carthage,2020,10,natural_gas,Total Consumption (T),Therms,-999.000,641.0
1,Jefferson,West Carthage,2020,10,natural_gas,Business Consumption (SC+O),Therms,-999.000,56.0
2,Jefferson,West Carthage,2020,10,natural_gas,Small Commercial Consumption (SC),Therms,9550.000,55.0
3,Jefferson,West Carthage,2020,10,natural_gas,Small Consumer Consumption (R+SC),Therms,35457.000,640.0
4,Jefferson,West Carthage,2020,10,natural_gas,Residential Consumption (R),Therms,25907.000,585.0
...,...,...,...,...,...,...,...,...,...
54155,Hamilton,Arietta,2019,10,electricity,Business Consumption (SC+O),MWh,29.177,55.0
54156,Hamilton,Arietta,2019,10,electricity,Small Consumer Consumption (R+SC),MWh,-999.000,644.0
54157,Hamilton,Arietta,2019,10,electricity,All Other Consumption (O),MWh,-999.000,1.0
54158,Hamilton,Arietta,2019,10,electricity,Small Commercial Consumption (SC),MWh,-999.000,54.0


,county_name,com_name,year,month,data_class,data_field_display_name,unit,value,number_of_accounts
8,Jefferson,West Carthage,2020,10,electricity,Total Consumption (T),MWh,NaN,918.0
9,Jefferson,West Carthage,2020,10,electricity,Business Consumption (SC+O),MWh,NaN,80.0
10,Jefferson,West Carthage,2020,10,electricity,Small Commercial Consumption (SC),MWh,246.554,73.0
11,Jefferson,West Carthage,2020,10,electricity,Small Consumer Consumption (R+SC),MWh,673.230,911.0
12,Jefferson,West Carthage,2020,10,electricity,Residential Consumption (R),MWh,426.676,838.0
...,...,...,...,...,...,...,...,...,...
54155,Hamilton,Arietta,2019,10,electricity,Business Consumption (SC+O),MWh,29.177,55.0
54156,Hamilton,Arietta,2019,10,electricity,Small Consumer Consumption (R+SC),MWh,NaN,644.0
54157,Hamilton,Arietta,2019,10,electricity,All Other Consumption (O),MWh,NaN,1.0
54158,Hamilton,Arietta,2019,10,electricity,Small Commercial Consumption (SC),MWh,NaN,54.0


,county_name,com_name,year,month,data_class,data_field_display_name,unit,value,number_of_accounts
0,Jefferson,West Carthage,2020,10,natural_gas,Total Consumption (T),Therms,-999.000,641.0
1,Jefferson,West Carthage,2020,10,natural_gas,Business Consumption (SC+O),Therms,-999.000,56.0
2,Jefferson,West Carthage,2020,10,natural_gas,Small Commercial Consumption (SC),Therms,9550.000,55.0
3,Jefferson,West Carthage,2020,10,natural_gas,Small Consumer Consumption (R+SC),Therms,35457.000,640.0
4,Jefferson,West Carthage,2020,10,natural_gas,Residential Consumption (R),Therms,25907.000,585.0
...,...,...,...,...,...,...,...,...,...
54155,Hamilton,Arietta,2019,10,electricity,Business Consumption (SC+O),MWh,29.177,55.0
54156,Hamilton,Arietta,2019,10,electricity,Small Consumer Consumption (R+SC),MWh,NaN,644.0
54157,Hamilton,Arietta,2019,10,electricity,All Other Consumption (O),MWh,NaN,1.0
54158,Hamilton,Arietta,2019,10,electricity,Small Commercial Consumption (SC),MWh,NaN,54.0


,county_name,com_name,year,month,data_class,data_field_display_name,unit,value,number_of_accounts
220,Jefferson,West Carthage,2019,12,natural_gas,CCA-Ineligible Customer Count (SC),count,17.000,0.0
221,Jefferson,West Carthage,2019,12,natural_gas,CCA-Ineligible Customer Count (R),count,106.000,0.0
222,Jefferson,West Carthage,2019,12,natural_gas,Total Consumption (T),Therms,93317.000,541.0
223,Jefferson,West Carthage,2019,12,natural_gas,Business Consumption (SC+O),Therms,16595.000,41.0
224,Jefferson,West Carthage,2019,12,natural_gas,Small Consumer Consumption (R+SC),Therms,93317.000,541.0
...,...,...,...,...,...,...,...,...,...
54155,Hamilton,Arietta,2019,10,electricity,Business Consumption (SC+O),MWh,29.177,55.0
54156,Hamilton,Arietta,2019,10,electricity,Small Consumer Consumption (R+SC),MWh,NaN,644.0
54157,Hamilton,Arietta,2019,10,electricity,All Other Consumption (O),MWh,NaN,1.0
54158,Hamilton,Arietta,2019,10,electricity,Small Commercial Consumption (SC),MWh,NaN,54.0


,county_name,com_name,year,month,data_class,data_field_display_name,unit,value,number_of_accounts
0,Jefferson,West Carthage,2020,10,natural_gas,Total Consumption (T),Therms,-999.000,641.0
1,Jefferson,West Carthage,2020,10,natural_gas,Business Consumption (SC+O),Therms,-999.000,56.0
2,Jefferson,West Carthage,2020,10,natural_gas,Small Commercial Consumption (SC),Therms,9550.000,55.0
3,Jefferson,West Carthage,2020,10,natural_gas,Small Consumer Consumption (R+SC),Therms,35457.000,640.0
4,Jefferson,West Carthage,2020,10,natural_gas,Residential Consumption (R),Therms,25907.000,585.0
...,...,...,...,...,...,...,...,...,...
54113,Hamilton,Arietta,2020,1,electricity,Business Consumption (SC+O),MWh,23.992,54.0
54114,Hamilton,Arietta,2020,1,electricity,Small Consumer Consumption (R+SC),MWh,NaN,642.0
54115,Hamilton,Arietta,2020,1,electricity,All Other Consumption (O),MWh,NaN,1.0
54116,Hamilton,Arietta,2020,1,electricity,Small Commercial Consumption (SC),MWh,NaN,53.0


In [ ]:
# use groupby to find the total number of country names 
group = df.groupby('county_name').size()
group

county_name
Franklin      7448
Fulton        4920
Genesee       7080
Greene        7048
Hamilton      3360
Herkimer     10040
Jefferson    13360
Tioga         4640
Tompkins      5480
Ulster        8840
dtype: int64